# Week 3 Assignment - Capstone-Project

#### Collecting data from the wikipedia page

##### Import packages required

In [12]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")

Imported!


##### Collecting Data from wikipedia page

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')
print("Done!")

Done!


##### Convert the obtained data here into Data Frame format

In [14]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
print(df)

    PostalCode       Borough           Neighborhood
0         None          None                   None
1          M1A  Not assigned           Not assigned
2          M2A  Not assigned           Not assigned
3          M3A    North York              Parkwoods
4          M4A    North York       Victoria Village
..         ...           ...                    ...
283        M8Z     Etobicoke              Mimico NW
284        M8Z     Etobicoke     The Queensway West
285        M8Z     Etobicoke  Royal York South West
286        M8Z     Etobicoke         South of Bloor
287        M9Z  Not assigned           Not assigned

[288 rows x 3 columns]


##### Drop unnecessary rows and missing values rows

In [15]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
print(df)

    PostalCode           Borough              Neighborhood
3          M3A        North York                 Parkwoods
4          M4A        North York          Victoria Village
5          M5A  Downtown Toronto              Harbourfront
6          M6A        North York          Lawrence Heights
7          M6A        North York            Lawrence Manor
..         ...               ...                       ...
282        M8Z         Etobicoke  Kingsway Park South West
283        M8Z         Etobicoke                 Mimico NW
284        M8Z         Etobicoke        The Queensway West
285        M8Z         Etobicoke     Royal York South West
286        M8Z         Etobicoke            South of Bloor

[210 rows x 3 columns]


##### Grouping similar ones by commas

In [16]:
def n_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df1 = grp.apply(n_list).reset_index(name='Neighborhood')
print(df1)

    PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                     Malvern, Rouge
1          M1C  Scarborough             Highland Creek, Port Union, Rouge Hill
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


##### Converting to csv

In [17]:
df1.to_csv('task1.csv', index=False)
df1.shape

(103, 3)

In [18]:
print(df1)

    PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                     Malvern, Rouge
1          M1C  Scarborough             Highland Creek, Port Union, Rouge Hill
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


In [19]:
!pip install geocoder
import geocoder

In [20]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [22]:
postal_codes = df['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [23]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [24]:
print(df)

    PostalCode           Borough              Neighborhood   Latitude  \
3          M3A        North York                 Parkwoods  43.723270   
4          M4A        North York          Victoria Village  43.723270   
5          M5A  Downtown Toronto              Harbourfront  43.661150   
6          M6A        North York          Lawrence Heights  43.662299   
7          M6A        North York            Lawrence Manor  43.811525   
..         ...               ...                       ...        ...   
282        M8Z         Etobicoke  Kingsway Park South West        NaN   
283        M8Z         Etobicoke                 Mimico NW        NaN   
284        M8Z         Etobicoke        The Queensway West        NaN   
285        M8Z         Etobicoke     Royal York South West        NaN   
286        M8Z         Etobicoke            South of Bloor        NaN   

     Longitude  
3   -79.451286  
4   -79.451286  
5   -79.391715  
6   -79.528195  
7   -79.195517  
..         ...  
282 

In [25]:
df[df.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.764765,-79.488094


In [27]:
df.to_csv('task2.csv', index=False)
print(df)

    PostalCode           Borough              Neighborhood   Latitude  \
3          M3A        North York                 Parkwoods  43.723270   
4          M4A        North York          Victoria Village  43.723270   
5          M5A  Downtown Toronto              Harbourfront  43.661150   
6          M6A        North York          Lawrence Heights  43.662299   
7          M6A        North York            Lawrence Manor  43.811525   
..         ...               ...                       ...        ...   
282        M8Z         Etobicoke  Kingsway Park South West        NaN   
283        M8Z         Etobicoke                 Mimico NW        NaN   
284        M8Z         Etobicoke        The Queensway West        NaN   
285        M8Z         Etobicoke     Royal York South West        NaN   
286        M8Z         Etobicoke            South of Bloor        NaN   

     Longitude  
3   -79.451286  
4   -79.451286  
5   -79.391715  
6   -79.528195  
7   -79.195517  
..         ...  
282 